In [22]:

config = {
  "audioConfig": {
    "audioEncoding": "LINEAR16",
    "pitch": 0,
    "speakingRate": 1
  },
  "input": {
    "text": "Senaste gången Riksbanken sänkte räntan var i juni i år då den sänktes med 0,25 procentenheter till 2 procent. Därefter, i augusti, valde man att lämna den oförändrad. Då sa Riksbanken att man såg en viss sannolikhet för ytterligare en sänkning i år."
  },
  "voice": {
    "languageCode": "sv-SE",
    "name": "sv-SE-Chirp3-HD-Achernar"
  }
}

url = "https://texttospeech.googleapis.com/v1beta1/text:synthesize"

In [23]:
import pandas as pd

df_sentences = pd.read_csv("../dataset/sentences.csv")

df_sentences = df_sentences.drop(columns=["sentiment"])

# Preview the dataframe
df_sentences.head()

,text
0,Flyttgiganten får kunder genom att de är billi...
1,Jag kontaktade BF Flytt & Stöd i god tid innan...
2,Har varit här en gång 2013 och minns inte anna...
3,Vi har använt kvalitetsflytt 2 ggr och har var...
4,"Dålig mäklare, ville inte ens visa lägenhtente..."


In [24]:
# Check the size of the dataset
print(f"Dataset contains {len(df_sentences)} sentences")
print(f"Columns: {list(df_sentences.columns)}")
print("\nFirst few sentences:")
for i, row in df_sentences.head().iterrows():
    print(f"{i}: {row['text'][:100]}...")

Dataset contains 62089 sentences
Columns: ['text']

First few sentences:
0: Flyttgiganten får kunder genom att de är billiga "säger de" men ack vad de lägger på extra timmar fö...
1: Jag kontaktade BF Flytt & Stöd i god tid innan min flytt. Trots det så kom flyttgubbarna inte i tid....
2: Har varit här en gång 2013 och minns inte annat än att det var gott och trevligt. Vi satt ute och ha...
3: Vi har använt kvalitetsflytt 2 ggr och har varit mycket nöjda med flytten. Väldigt smidigt. Rekommen...
4: Dålig mäklare, ville inte ens visa lägenhtenten tills jag tjatade mig till det, seg i budgivningen o...


In [25]:
import os
from google.cloud import texttospeech

# Set up authentication - Option 1: Use Application Default Credentials
# First, authenticate using: gcloud auth application-default login
# Or set GOOGLE_APPLICATION_CREDENTIALS environment variable

# Initialize the client
client = texttospeech.TextToSpeechClient()

# Create the synthesis input, voice, and audio config from the config dictionary
synthesis_input = texttospeech.SynthesisInput(text=config["input"]["text"])
voice = texttospeech.VoiceSelectionParams(
    language_code=config["voice"]["languageCode"],
    name=config["voice"]["name"]
)
audio_config = texttospeech.AudioConfig(
    audio_encoding=texttospeech.AudioEncoding[config["audioConfig"]["audioEncoding"]],
    pitch=config["audioConfig"]["pitch"],
    speaking_rate=config["audioConfig"]["speakingRate"]
)



# Save the audio content to a file
# with open("../dataset/audios/audio_0.mp3", "wb") as out:
#     out.write(response.audio_content)
#     print("Audio content written to file '../dataset/audios/audio_0.mp3'")

# response

E0000 00:00:1758631530.999839 1715321 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [ ]:
# Import additional libraries needed for processing
import pyarrow.parquet as pq
import pyarrow as pa
from tqdm import tqdm
import time
import random

def generate_audio_for_text(text):
    
    voices = [
        "sv-SE-Chirp3-HD-Achernar",
        "sv-SE-Chirp3-HD-Achird",
        "sv-SE-Chirp3-HD-Algenib",
        "sv-SE-Chirp3-HD-Zubenelgenubi",
        "sv-SE-Chirp3-HD-Zephyr",
        "sv-SE-Chirp3-HD-Vindemiatrix",
        "sv-SE-Chirp3-HD-Umbriel",
        "sv-SE-Chirp3-HD-Orus",
        "sv-SE-Chirp3-HD-Iapetus",
        "sv-SE-Chirp3-HD-Gacrux"
    ]
    
    try:
        voice = texttospeech.VoiceSelectionParams(
            language_code=config["voice"]["languageCode"],
            name=voices[random.randint(0, len(voices)-1)]
        )
        synthesis_input = texttospeech.SynthesisInput(text=text)
        response = client.synthesize_speech(
            input=synthesis_input, 
            voice=voice, 
            audio_config=audio_config
        )
        return response.audio_content
    except Exception as e:
        print(f"Error generating audio for text: {text[:50]}... Error: {str(e)}")
        return None

print("Audio generation function created successfully!")

Audio generation function created successfully!


In [ ]:
# Process sentences and generate audio
# Let's start with a smaller subset for testing (first 10 sentences)
# You can change this to len(df_sentences) to process all sentences

# For testing, let's use just the first 10 sentences
start_from = 100
# df_sample = df_sentences.head(test_size).copy()
df_sample = df_sentences[start_from:].copy()

print(f"Processing {len(df_sample)} sentences...")



# Process each sentence
for idx, row in tqdm(df_sample.iterrows(), total=len(df_sample), desc="Generating audio"):
    text = row['text']
    
    # Generate audio for this text
    audio_bytes = generate_audio_for_text(text)
    if(audio_bytes):
        with(open(f"../dataset/audios/audio_{idx}.wav", "wb") as out_file):
            out_file.write(audio_bytes)
    else:
        print(f"Failed to generate audio for index {idx}, text: {text[:50]}...")
    
    # Small delay to avoid hitting API rate limits
    time.sleep(0.1)

Processing 62089 sentences...


Generating audio:   0%|          | 104/62089 [03:04<30:30:14,  1.77s/it]


KeyboardInterrupt: 